# 멀티 에이전트 협업 네트워크(Multi-Agent Collaboration Network)

**Multi-Agent Collaboration Network**는 여러 AI 에이전트들이 협력하여 복잡한 문제를 해결하는 시스템입니다.

### 핵심 개념

**기본 구조**
- 각 에이전트가 특정 역할이나 전문성을 가지고 있음
- 에이전트들이 서로 통신하고 정보를 공유하며 협업
- 중앙 조정자가 있거나, 분산형으로 자율적으로 협력

**주요 특징**
- **분업**: 복잡한 작업을 여러 에이전트가 나누어 처리
- **상호작용**: 에이전트 간 메시지 전달, 결과 공유, 피드백 교환
- **시너지**: 단일 에이전트보다 더 나은 성과 달성

**활용 예시**
- 코딩 작업: 기획자, 개발자, 테스터 에이전트가 협력
- 연구 분석: 데이터 수집, 분석, 보고서 작성을 각각 담당
- 고객 서비스: 문의 분류, 답변 생성, 품질 검증 에이전트 협업

**장점**
- 복잡한 문제를 더 효과적으로 해결
- 각 에이전트의 전문성 활용
- 확장성과 유연성

- 참고 자료: https://wikidocs.net/270689
- 관련 논문: https://arxiv.org/abs/2308.08155
- Multi-agent systems 개념: https://langchain-ai.github.io/langgraph/concepts/multi_agent/

## 환경 설정

In [1]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv("../.env", override=True)


def _set_env(var: str):
    env_value = os.environ.get(var)
    if not env_value:
        env_value = getpass.getpass(f"{var}: ")

    os.environ[var] = env_value


_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"
_set_env("OPENAI_API_KEY")

## 상태 정의

In [3]:
from typing import Annotated
from langgraph.graph.message import MessagesState


class State(MessagesState):
    sender: Annotated[str, "마지막 메시지의 발신자"]

## 도구 정의

In [4]:
from langchain_core.tools import Tool
from langchain_tavily import TavilySearch
from langchain_experimental.utilities import PythonREPL

web_search_tool = TavilySearch(max_results=3)

python_repl = PythonREPL()
python_repl_tool = Tool(
    "python_repl",
    description="파이썬 셸입니다. 파이썬 명령어를 실행하는 데 사용하세요. 입력은 유효한 파이썬 명령어여야 합니다. 값의 출력을 확인하려면 `print(...)`로 출력해야 합니다.",
    func=python_repl.run,
)

## Research Agent

웹 검색 도구를 사용하여 연구에 필요한 정보를 리서치하는 에이전트입니다.

In [ ]:
from langgraph.prebuilt import create_react_agent


research_agent = create_react_agent(
    model="openai:gpt-4.1-mini",
    tools=[web_search_tool],
    prompt="팀 어시스턴트로서 인터넷 리서치를 통해 연구에 필요한 정보를 수집하고, 제공된 도구로 질문에 답하세요."
    "부족한 부분은 다른 어시스턴트가 이어받습니다."
    "차트 생성은 전담 담당자와 협력해 진행합니다."
    "최종 결과 앞에 “FINAL ANSWER”를 붙여 알리세요.",
)

In [6]:
from langchain_core.messages import HumanMessage


def research_node(state: State):
    response = research_agent.invoke({"messages": state["messages"]})
    last_message = response["messages"][-1]["content"]
    return {"messages": [HumanMessage(content=last_message, name="researcher agent")]}

## Chart Generator Agent

파이썬 코드를 사용하여 차트를 생성하는 에이전트입니다.

In [ ]:
chart_generator_agent = create_react_agent(
    model="openai:gpt-4.1-mini",
    tools=[python_repl_tool],
    prompt="당신은 파이썬 코드 작성 도구를 사용해 연구용 차트를 생성하는 전담 에이전트입니다."
    "연구원 동료와 협력하여 필요한 차트만 작성하세요."
    "차트를 생성할 때는 반드시 한글 글꼴 코드를 코드에 포함해야 합니다."
    "최종 결과 앞에 “FINAL ANSWER”를 붙여 알리세요.",
)

In [ ]:
def chart_generator_node(state: State):
    response = research_agent.invoke({"messages": state["messages"]})
    last_message = response["messages"][-1]["content"]
    return {
        "messages": [HumanMessage(content=last_message, name="chart generator agent")]
    }

## 그래프 작성

In [ ]:
from typing import Literal
from langgraph.graph import StateGraph, END


def should_continue(state: State) -> Literal["continue", END]:
    last_message = state["messages"][-1]["content"]
    if "FINAL ANSWER" in last_message:
        return END
    else:
        return "continue"


builder = StateGraph(State)
builder.add_noe(research_node)
builder.add_noe(chart_generator_node)

builder.add_conditional_edges(
    research_node.__name__,
    should_continue,
    {
        "continue": chart_generator_node.__name__,
        END: END,
    },
)
